# Square Matrix Generator
The purpose of this notebook is to write (and clearly comment) a function for finding the root of an inverse variance matrix and verifying it via matrix inversion, as well as timing the methods to see if they're viable for use for a 1000x1000 matrix.

Here's the initial test matrices, as well as tests for ensuring they're symmetric (since they're real, this implies their Hermitian, which will make solving for eigenvectors and eigenvalues easier):

In [31]:
import numpy as np
import scipy as sp
import scipy.linalg
import time

#setting the size of the arrays for testing, will start small for testing and then will scale up for timing
N = 1000

#sigmas exist on a uniform distribution from [100, 300]

#pure diagonal matrix with equal values
sigma1 = np.random.random()*200. + 100.
V1 = sigma1 * np.identity(N)
#checks to ensure symmetry (and hermicity since all real)
print np.allclose(V1,np.ndarray.transpose(V1))

#diagonal matrix with different elements
sigma2 = np.random.rand(N)*200 +100
V2 = sigma2 * np.identity(N)
print np.allclose(V2,np.ndarray.transpose(V2))




True
True


Let's write the method for finding the matrix square roots. The general idea is that we can decompose a matrix $V$ as:

$V = \sum_d^D \lambda_d \vec{U} \vec{U}^T$
or
$V^{-1} = \sum_d^D \frac{1}{\lambda_d} \vec{U} \vec{U}^T$

Where $\lambda_d$ are the eigenvalues, and $\vec{U}_d$ are the corresponding eigenvectors. This can be further decomposed as:

$V^{-1} = \sum_d^D \frac{1}{\sqrt{\lambda_d}}  \vec{U} \cdot \vec{U}^T \frac{1}{\sqrt{\lambda_d}}$

If we set the definition $w_d \equiv \frac{1}{\sqrt{\lambda_d}} \vec{U}_d ^T$, then we can write:

$V^{-1}= w^{T} \cdot w$

So inverting the matrix becomes a simple case of finding the eigenvectors and eigenvalues, constructing $w$, and transposing and taking the dot product.

My function below calculates $w$:


In [47]:
#computes matrix square root of inverse variance matrix provided variance matrix
def matrixsqrt(V, label="matrix"):
    N = len(V[0])
    #extracts eigenvalues and eigenvectors (bottleneck!)
    eigs, eigvecs = sp.linalg.eigh(V)
    wt = np.empty([N,N]) #square root matrix (transposed)
    logdet = 0.
    start = time.time()
    for i in range(N):
        #sets each column in our transposed square root to the eigenvalue scaled by 1/sqrt(eig)
        wt[:,i] = (1./np.sqrt(eigs[i])) * eigvecs[:,i]
        logdet += np.log(2 * np.pi * eigs[i])
        #transposes the result
    w = np.ndarray.transpose(wt)
    end = time.time()
    print("Time elapsed for " + label + " is: " + str(end-start) + "s")
    return w, logdet


Let's test this with our two test functions. Taking the dot product of the matrix square root and it's transpose should produce $V^{-1}$, so the dot product of that with $V$ should  produce the identity matrix, so this is our test:

In [48]:
w1, logdet1 = matrixsqrt(V1, label="V1")
print np.dot(V1, np.dot(np.ndarray.transpose(w1), w1))


w2, logdet2 = matrixsqrt(V2, label="V2")
print np.dot(V2, np.dot(np.ndarray.transpose(w2), w2))


Time elapsed for V1 is: 0.0217518806458s
[[ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  1.  0.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  0.  1.]]
Time elapsed for V2 is: 0.0860199928284s
[[ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  1.  0.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  0.  1.]]


Great, it looks like we're working. Let's try the third sample variance matrix, which is our second test matrix $V_2$ added with a kernal matrix $K$, defined as:

$K_{ab} = A \cdot exp\left({-\frac{1}{2}\frac{(t_a-t_b)^2}{\tau^2}}\right)$

Where $A$ and $\tau$ are parameters to be fit corresponding to noise and time correlation, but for testing purposes, we'll set them to $A=200$ and $\tau = 30s$.

In [43]:
#opening data
data1 = "/home/chris/Documents/QPP/SolarFlareGPs/data/121022782_ctime_lc.txt"
t1, I1 = np.loadtxt(data1, unpack=True)

A = 300
tau = 30

N = len(t1)

K = np.empty([N,N])
for i in range(N):
    for j in range(N):
        K[i][j] = A * np.exp(-0.5 * np.power(((t1[i]-t1[j])/(tau)),2))

sigma2 = np.random.rand(N)*200 +100
V2 = sigma2 * np.identity(N)

V3 = V2 + K
print np.allclose(V3,np.ndarray.transpose(V3))

True


And now, let's try to find the matrix square root of this guy:

In [44]:
w3, logdet3 = matrixsqrt(V3, label="V3")
result = np.dot(V3, np.dot(np.ndarray.transpose(w3), w3))

np.allclose(result,np.identity(N))

Time elapsed for V3 is: 0.0280699729919s
[[  1.00000000e+00   3.56022598e-15   1.39643727e-15 ...,   1.33533296e-17
   -1.43870228e-17   1.22235150e-17]
 [  1.07143671e-15   1.00000000e+00  -9.33988884e-15 ...,   5.00590339e-16
   -1.50004220e-16   4.57205098e-17]
 [  1.02392639e-16  -9.90690803e-15   1.00000000e+00 ...,  -2.24179169e-16
    2.31696194e-16  -6.98397777e-18]
 ..., 
 [ -9.05821912e-17   6.48604932e-16   2.36135859e-16 ...,   1.00000000e+00
    1.60129489e-16   3.34662635e-16]
 [ -1.03769326e-16   9.92893692e-17   6.33093624e-16 ...,  -3.77956707e-16
    1.00000000e+00  -4.91080390e-16]
 [ -8.13016862e-17   3.35412621e-16   3.59260381e-16 ...,   2.51365828e-16
    3.61489495e-16   1.00000000e+00]]
